In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/cnn_proj_data/dataset/archive_data/ckextended.csv')

In [ ]:
print(data.head())
print(data.dtypes)

   emotion                                             pixels     Usage
0        6  36 39 35 25 19 11 8 7 3 13 15 9 21 57 75 90 10...  Training
1        6  88 74 19 4 5 5 3 12 8 21 15 21 15 18 24 29 32 ...  Training
2        6  9 2 4 7 1 1 1 0 7 29 49 76 115 141 156 169 177...  Training
3        6  104 106 108 104 95 50 60 61 58 83 126 133 139 ...  Training
4        6  68 72 67 67 6 2 1 1 1 1 1 14 24 24 38 65 79 94...  Training
emotion     int64
pixels     object
Usage      object
dtype: object


In [ ]:
def process_pixels(pixels):
    if isinstance(pixels, str):
        pixels = np.array(pixels.split(), dtype='float32')
    return pixels / 255.0

data['pixels'] = data['pixels'].apply(process_pixels)

# Extract features and labels
X = np.stack(data['pixels'].values)
y = data['emotion'].values

# Check unique values in 'emotion' column
print(f"Unique emotion labels: {np.unique(y)}")

# Reshape the data
X = X.reshape(X.shape[0], 48, 48, 1)  #images are 48x48 pixels

# One-hot encode labels
from keras.utils import to_categorical
num_classes = 8
y = to_categorical(y, num_classes=num_classes)

# Split the data into training and testing sets
train_indices = data['Usage'] == 'Training'
test_indices = data['Usage'] == 'PublicTest'

X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]


print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Unique emotion labels: [0 1 2 3 4 5 6 7]
Training set size: 734
Test set size: 91


In [ ]:
model = Sequential()

#1st convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#2nd convolutional layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#3rd convolutional layer
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())

# Fully connected layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 23, 23, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 64)        0

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
batch_size = 32
epochs = 50

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=1)


Epoch 1/50
23/23 [==============================] - 3s 115ms/step - loss: 1.3242 - accuracy: 0.6458 - val_loss: 1.3084 - val_accuracy: 0.6484
Epoch 2/50
23/23 [==============================] - 2s 109ms/step - loss: 1.3221 - accuracy: 0.6458 - val_loss: 1.2968 - val_accuracy: 0.6484
Epoch 3/50
23/23 [==============================] - 4s 178ms/step - loss: 1.3363 - accuracy: 0.6458 - val_loss: 1.2911 - val_accuracy: 0.6484
Epoch 4/50
23/23 [==============================] - 3s 132ms/step - loss: 1.3112 - accuracy: 0.6458 - val_loss: 1.2967 - val_accuracy: 0.6484
Epoch 5/50
23/23 [==============================] - 3s 110ms/step - loss: 1.3338 - accuracy: 0.6458 - val_loss: 1.2985 - val_accuracy: 0.6484
Epoch 6/50
23/23 [==============================] - 3s 109ms/step - loss: 1.3304 - accuracy: 0.6458 - val_loss: 1.2996 - val_accuracy: 0.6484
Epoch 7/50
23/23 [==============================] - 2s 108ms/step - loss: 1.3274 - accuracy: 0.6458 - val_loss: 1.2954 - val_accuracy: 0.6484
Epoch 

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


Test loss: 1.2919360399246216
Test accuracy: 0.6483516693115234
